In [2]:
import csv, sys
filename = 'plate_1_dummy.csv'
with open(filename, 'rb') as f:
    reader = csv.reader(f)
    try:
        for row in reader:
            print row
    except csv.Error as e:
        sys.exit('file %s, line %d: %s' % (filename, reader.line_num, e))

['2860', '990', '1100', '1130', '1660', '1340', '1690', '1850', '3460', '6700', '13490', '11270']
['2320', '830', '2200', '1020', '980', '2530', '1090', '1200', '1090', '2070', '950', '1020']
['980', '920', '6550', '980', '1130', '1200', '1280', '19130', '2640', '5640', '13260', '12190']
['900', '900', '960', '1040', '1210', '1340', '2160', '2300', '3050', '5050', '13990', '15230']
['900', '1860', '3140', '1220', '1350', '5320', '1540', '1840', '3220', '8620', '17060', '15270']
['900', '1040', '2410', '1120', '1380', '1360', '1600', '1960', '3250', '6860', '16870', '10500']
['990', '3590', '1030', '1330', '1050', '34590', '1290', '1700', '2900', '8720', '16150', '11000']
['900', '880', '1080', '1040', '1610', '1560', '1320', '1660', '2920', '5850', '14270', '14430']


In [175]:
rows = 8
cols = 12

def read_plate_csv(filename):
    import csv, sys
    
    csv_list = list()
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        try:
            for row in reader:
                csv_list.append(row)
        except csv.Error as e:
            # TODO handle
            print 'file %s, line %d: %s' % (filename, reader.line_num, e)
            
    return csv_list

def display_read_plate_csv():
    dummy_well_1 = read_read_plate_csv('plate_1_dummy.csv')    
    for row in dummy_well_1:
        print '\nROW:'
        for well in row:
            print '\t' + well
            
#display_read_well_csv()

def get_well_info(filename='elisawells.csv',
                 rows=8):
    import csv, sys
    
    rows = rows
    row_count = 0
    col_count = 0
    
    # create list of lists
    col_list = list()
    
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        try:
            col_count_list = list()
            
            for row in reader:
                row_value = int(row[0])
                
                if row_count >= rows:
                    row_count = 0
                    col_count = col_count + 1
                    #print col_count_list
                    col_list.append(col_count_list)
                    col_count_list = list()
                
                row_count = row_count + 1
                col_count_list.append(row_value)
                
                #print '%i\t%i\t%i' % (row_count, col_count, row_value)
            
            if col_count_list:
                col_list.append(col_count_list)
            
        except csv.Error as e:
            # TODO handle
            print 'file %s, line %d: %s' % (filename, reader.line_num, e)
    
    col_resize = list()
    
    for i in range(rows):
        tmp = list()
        
        for j in range(len(col_list)):
            tmp.append(col_list[j][i])
            
        col_resize.append(tmp)
    
    return col_resize

well_map = get_well_info()
plate = read_plate_csv('plate_1_dummy.csv')

# get well xy pos for num
# for i in range(len(well_map)):
#     for j in well_map[i]:
#         print j
#         well_pos = j-1
#         well_col = abs(well_pos/rows)+1
#         well_row = abs(well_pos%rows)
        
#         print well_col
#         print well_row

def get_mapped_plate(filename='elisawells.csv',
                 rows=8, cols=12, plate=plate):
    import csv, sys
    
    row_count = 0
    col_count = 0
    
    mapped_plate = list()
       
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        col_count_list = list()

        for row in reader:
            row_value = int(row[0])
    
            well_pos = row_value
            well_col = abs(well_pos/rows)
            well_row = abs(well_pos%rows)-1
            
            mapped_plate.append((row_value, well[well_row][well_col]))
            
    return mapped_plate
            
get_mapped_plate()

[(20, '960'),
 (27, '980'),
 (28, '1040'),
 (61, '1840'),
 (5, '900'),
 (26, '1020'),
 (29, '1220'),
 (75, '5640'),
 (38, '1380'),
 (67, '2640'),
 (71, '2900'),
 (76, '5050'),
 (6, '900'),
 (57, '1850'),
 (66, '1090'),
 (73, '6700'),
 (9, '990'),
 (11, '920'),
 (51, '1280'),
 (78, '6860'),
 (23, '1030'),
 (30, '1120'),
 (41, '1340'),
 (46, '1360'),
 (33, '1660'),
 (54, '1600'),
 (63, '1700'),
 (64, '2920'),
 (32, '1610'),
 (44, '1340'),
 (65, '3460'),
 (74, '2070'),
 (2, '2320'),
 (3, '980'),
 (25, '1130'),
 (39, '1050'),
 (13, '1860'),
 (37, '1350'),
 (50, '1090'),
 (80, '14270'),
 (1, '2860'),
 (56, '1660'),
 (68, '3050'),
 (79, '8720'),
 (22, '2410'),
 (36, '1210'),
 (60, '2300'),
 (72, '5850'),
 (35, '1130'),
 (43, '1200'),
 (53, '1540'),
 (58, '1200'),
 (10, '830'),
 (14, '1040'),
 (62, '1960'),
 (77, '8620'),
 (24, '1040'),
 (31, '1330'),
 (52, '2160'),
 (69, '3220'),
 (15, '3590'),
 (21, '3140'),
 (48, '1320'),
 (59, '19130'),
 (4, '900'),
 (16, '1080'),
 (17, '1100'),
 (34, '98

In [134]:
abs(19 / 8)

2

In [135]:
19%8

3